In [437]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs

In [384]:
# download data from the website
def download_page(url, method_bs=True):
    res = requests.get(url)
#     if method_bs == True:
#         return bs(res.text, 'html.parser')
    return res.content

def download_pages(list_url, method_bs=True):
    data = []
    for link in links:
        data.append(download_page(link, method_bs=True))
    return data

In [385]:
import re
def scrape_df(data_page):
    # get the email
    css_email = '::attr(href)'
    re_email = '[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+'

    program_email = ''
    for i in data_page:
        print(i)
        print('1 --------------')
#         print(re.findall(re_email, i))
#     return program_email

In [386]:
from scrapy import Selector

def get_selector(text, css_selector):
    sel = Selector(text=text)
    extraction = sel.css(css_selector).extract()
    return extraction

def get_name(text):
    name = get_selector(text, '.title::text')
    try:
        return list(map(lambda x:x.split(':'), name))[0][0].strip()
    except:
        return name

def get_phone(text):
    phone = get_selector(text, 'p:nth-child(4)::text')
    try:
        return list(map(lambda x:x.split(':'), phone))[0][1].strip()
    except:
        return phone

def get_email(text):
    email = get_selector(text, 'p:nth-child(5)::text')
    try:
        return list(map(lambda x:x.split(':'), email))[0][2].strip()
    except:
        return email

def get_site(text):
    site = get_selector(text, 'p:nth-child(8)::text')
    try:
        return list(map(lambda x:x.split(':'), site))[0][1].strip()
    except:
        return site

In [401]:
def extract_data(text, pattern):
    emails = list(set(re.findall(pattern, str(text))))
    return ', '.join(emails)

def get_email(text):
    return extract_data(text, '[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+')

In [380]:
email_pattern = '[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+'
website_pattern = r'Site:.(\w+.)'

In [403]:
extract_data(sec, website_pattern)

''

In [347]:
links = [
    'http://www.cofen.gov.br/conselho-regional-de-enfermagem-do-rio-grande-do-sul_4431.html',
    'http://www.cofen.gov.br/conselho-regional-de-enfermagem-do-amazonas_4411.html',
    'http://www.cofen.gov.br/conselho-regional-de-enfermagem-do-distrito-federal_4415.html'
       ]

In [298]:
data = download_pages(links, method_bs=False)

In [348]:
def get_all_entities(text):
    # get data
    name = get_name(text)
    phone = get_phone(text)
    email = get_email(text)
    site = get_site(text)

    # creating a dataframe
    df = pd.DataFrame([name, phone, email, site]).T
    df.columns = ['name', 'phone', 'email', 'website']
    return df

In [349]:
a = get_all_entities(data[1])

In [350]:
a

,name,phone,email,website
0,Conselho Regional de Enfermagem do Amazonas,8h – 16h,secretariaexecutiva@corenam.gov.br,[]


In [317]:
l = []
for i in range(len(data)):
    entities = get_all_entities(data[i])
    l.append(entities)

In [318]:
pd.concat(l)

,name,phone,email,website
0,Conselho Regional de Enfermagem do Rio Grande ...,(51) 3378-5500,"sac@portalcoren-rs.gov.br, sic@portalcoren-rs....",www.portalcoren-rs.gov.br
0,Conselho Regional de Enfermagem do Amazonas,8h – 16h,secretariaexecutiva@corenam.gov.br,[]
0,Conselho Regional de Enfermagem do Distrito Fe...,"[Segunda a sexta, das 7h às 18h – exclusivo pa...",secretariagabinete@coren-df.org.br,[]


## Secretarias de Saúde

In [557]:
link = 'https://www.conass.org.br/secretarios-estaduais-de-saude/'

In [558]:
sec = download_page(link)

In [585]:
for i in get_selector(sec, 'strong::text'):
    if len(i) > 3:
        print(i)

RELAÇÃO DAS SECRETARIAS ESTADUAIS DE SAÚDE
Alexandre Ayres
Juan Mendes da Silva
Fábio Vilas Boas
Carlos Roberto Martins Rodrigues Sobrinho
Osney Okumoto
Nésio Fernandes de Medeiros Junior
Carlos Eduardo de Oliveira Lula
Gilberto Figueiredo
Geraldo Resende
Rômulo Rodovalho
Geraldo Antônio de Medeiros
Carlos Alberto Gebrim Preto
André Longo
Alex da Silva Bousquet
Cipriano Maia
Arita Gilda Hübner Bergmann
Marcelo Lima Lopes
André Motta Ribeiro
Mércia Feitosa
Luiz Edgar Leão Tolini


In [567]:
data = get_selector(sec,'p::text')

In [568]:
def get_data(word, list_strings):
    l = []
    for string in list_strings:
        if word in string:
            l.append(string)
    return l

In [586]:
phone = get_data('Facebook', data)
clean_phones = list(map(lambda x:x.replace('Telefone: ', 'Phone: +55'), phone))

In [587]:
for i in clean_phones:
    print(i)